In [19]:
import abc
import numpy as np
from collections import namedtuple
from scipy.stats import norm


# Reference: https://github.com/dedwards25/Python_Option_Pricing/blob/master/GBS.ipynb

Greeks = namedtuple("greeks", "option_value delta gamma theta vega rho")

In [20]:
class GeneralisedEuropeanBlackScholes(abc.ABC):
    """
    Args:
        F_t: price of the underlying
        X_t: underlying strike price
        t:   time to expiration
        vol: implied volatility
        r:   risk-free rate
        q:   (potential) dividend payment
        b:   (potential) cost of carry
        
    Returns:
        option_value: calculated price of the option
        delta:        1st derivative of value wrt. the price of the underlying (F_t)
        gamma:        2nd derivative of value wrt. the price of the underlying (F_t)
        theta:        1st derivative of value wrt. time to expiration (t)
        vega:         1st derivative of value wrt. implied volatility (vol)
        rho:          1st derivative of value wrt. risk-free rate (r)
    """
    
    def __init__(self, name="GBS"):
        self.name = name
        
    def call(self, F_t, X_t, t, r, b, vol):
        # Pre-calculations
        sqrt_t = np.sqrt(t)
        d1, d2 = self._distribution_parameters(F_t, X_t, t, r, b, vol, sqrt_t)
        
        # Fair value price
        option_value = F_t * np.exp((b - r)*t) * norm.cdf(d1) - X_t * np.exp(-r*t) * norm.cdf(d2)

        # Greeks
        delta = np.exp((b - r)*t) * norm.cdf(d1)
        gamma = np.exp((b - r)*t) * norm.pdf(d1) / (F_t * vol * sqrt_t)
        theta = -(F_t * vol * np.exp((b - r)*t) * norm.pdf(d1)) / (2*sqrt_t) - (b - r) * F_t * np.exp((b - r)*t) * norm.cdf(d1) - r * X_t * np.exp(-r * t) * norm.cdf(d2)
        vega = np.exp((b - r)*t) * F_t * sqrt_t * norm.pdf(d1)
        rho = X_t * t * np.exp(-r*t) * norm.cdf(d2)
        
        return Greeks(option_value, delta, gamma, theta, vega, rho)
        
    def put(self, F_t, X_t, t, r, b, vol):
        # Pre-calculations
        sqrt_t = np.sqrt(t)
        d1, d2 = self._distribution_parameters(F_t, X_t, t, r, b, vol, sqrt_t)
        
        # Fair value price
        option_value = X_t * np.exp(-r*t) * norm.cdf(-d2) - (F_t * np.exp((b - r)*t) * norm.cdf(-d1))
        
        # Greeks
        delta = -np.exp((b - r)*t) * norm.cdf(-d1)
        gamma = np.exp((b - r)*t) * norm.pdf(d1) / (F_t * vol * sqrt_t)
        theta = -(F_t * vol * np.exp((b - r)*t) * norm.pdf(d1)) / (2*sqrt_t) + (b - r) * F_t * np.exp((b - r)*t) * norm.cdf(-d1) + r * X_t * np.exp(-r*t) * norm.cdf(-d2)
        vega = np.exp((b - r)*t) * F_t * sqrt_t * norm.pdf(d1)
        rho = -X_t * t * np.exp(-r*t) * norm.cdf(-d2)

        return Greeks(option_value, delta, gamma, theta, vega, rho)
    
    @staticmethod
    def _distribution_parameters(F_t, X_t, t, r, b, vol, sqrt_t):
        # Static method allows for cleaner use of scipy.optimize.
        d1 = (np.log(F_t/X_t) + (b + np.square(vol)/2) * t) / (vol * sqrt_t)
        d2 = d1 - vol * sqrt_t
        
        return d1, d2
    
    @property
    def cost_of_carry(self, *args, **kwargs):
        raise NotImplementedError

In [21]:
class Black76Eur(GeneralisedEuropeanBlackScholes):
    """
    Args:
        F_t: price of the underlying
        X_t: underlying strike price
        t:   time to expiration
        vol: implied volatility
        r:   risk-free rate
        q:   (potential) dividend payment
        b:   (potential) cost of carry
        
    Returns:
        option_value: calculated price of the option
        delta:        1st derivative of value wrt. the price of the underlying (F_t)
        gamma:        2nd derivative of value wrt. the price of the underlying (F_t)
        theta:        1st derivative of value wrt. time to expiration (t)
        vega:         1st derivative of value wrt. implied volatility (vol)
        rho:          1st derivative of value wrt. risk-free rate (r)
    """
    
    def __init__(self):
        super().__init__(name="Black76")
                
    def call(self, F_t, X_t, t, r, vol):
        b = self.cost_of_carry
        return super().call(F_t=F_t, X_t=X_t, t=t, r=r, b=b, vol=vol)
            
    def put(self, F_t, X_t, t, r, vol):
        b = self.cost_of_carry
        return super().put(F_t=F_t, X_t=X_t, t=t, r=r, b=b, vol=vol)
    
    @property
    def cost_of_carry(self):
        # No cost of carry on the underlying (futures contract).
        return 0

In [22]:
F_t = 50
X_t = 55
t = 1/12
r = 0.04
vol = 0.20


black76 = Black76Eur()
black76.call(F_t=F_t, X_t=X_t, t=t, r=r, vol=vol), black76.put(F_t=F_t, X_t=X_t, t=t, r=r, vol=vol)

(greeks(option_value=0.06213200829622734, delta=0.05223228374301364, gamma=0.03696551723344639, theta=-1.8457905813404705, vega=1.540229884726933, rho=0.2124568482378712),
 greeks(option_value=5.04549308856884, delta=-0.9444399323115097, gamma=0.03696551723344639, theta=-1.6464561381295657, vega=1.540229884726933, rho=-4.355624142012027))

In [23]:
def implied_volatility_black76_european(option_type, F_t, X_t, t, r, option_price_t, precision, max_num_iterations):
    black76 = Black76Eur()
    
    # Map option type to a function to be optimised.
    pricing_funcs = {
        "put": black76.put,
        "call": black76.call,
    }
    
    b = black76.cost_of_carry
    
    return _implied_volatility_gbs(
        option_value_fn=pricing_funcs[option_type],
        option_type=option_type,
        F_t=F_t,
        X_t=X_t,
        t=t,
        r=r,
        b=b,
        option_price_t=option_price_t,
        precision=precision,
        max_num_iterations=max_num_iterations,
    )    

In [24]:
def _approx_implied_volatility(option_type, F_t, X_t, t, r, b, option_price_t):
    """
    Choose an initial value from which to start a search function, e.g. Newton Raphson.
    From: Brenner & Subrahmanyam (1988), Feinstein (1988).
    """
    a = np.sqrt(2.0*np.pi) / (F_t*np.exp((b - r)*t) + X_t*np.exp(-r*t))
    
    # Calls
    payoff = F_t*np.exp((b - r)*t) - X_t*np.exp(-r*t)

    # Puts
    if option_type == "put":
        payoff *= -1

    b = option_price_t - payoff / 2.0
    c = (payoff ** 2) / np.pi
    v = (a * (b + np.sqrt(b ** 2 + c))) / np.sqrt(t)

    return v

In [25]:
def _implied_volatility_gbs(
    option_value_fn,
    option_type,
    F_t,
    X_t,
    t,
    r,
    b,
    option_price_t,
    precision=0.0001,
    max_num_iterations=100,
    vol_min=0.0,
    vol_max=10.0,  # `10` represents 1000%, since `1` represents 100%.
):
    """
    Calculate Implied Volatility with a Newton Raphson search
    
    Args:
        option_value_fn:
        option_type:
        F_t:
        X_t:
        t:
        r:
        b:
        option_price_t: known market price of the option (put or call) at time t
        precision:
        max_num_iterations:
        vol_min: volatility must be positive, some formulae may need a small positive minimum, e.g. 0.5%, i.e. 0.005.
        vol_max: not striclty necessary. Prevents erroneous choices, e.g. 20% should be 0.2, not `20`. 
    """

    # Estimate starting volatility, making sure it is allowable range
    implied_vol = _approx_implied_volatility(option_type=option_type, F_t=F_t, X_t=X_t, t=t, r=r, b=b, option_price_t=option_price_t)
    implied_vol = max(vol_min, min(vol_max, implied_vol))

    
    # Back-calculate the value of the option at the current estimate of volatility.
    greeks = option_value_fn(F_t=F_t, X_t=X_t, t=t, r=r, vol=implied_vol)
    option_price_estimate = greeks.option_value
    vega = greeks.vega

    # Iterate until the value, given estimated implied vol, is near to the known option price.
    min_valuation_error = abs(option_price_t - option_price_estimate)

    # Newton-Raphson Search
    num_iterations = 0
    while precision <= abs(option_price_t - option_price_estimate) <= min_valuation_error and num_iterations < max_num_iterations:

        implied_vol = implied_vol - (option_price_estimate - option_price_t) / vega
        if (implied_vol > vol_max) or (implied_vol < vol_min):
            print("Volatility out of bounds")
            break

        greeks = option_value_fn(F_t=F_t, X_t=X_t, t=t, r=r, vol=implied_vol)
        option_price_estimate = greeks.option_value
        vega = greeks.vega
        
        min_valuation_error = min(abs(option_price_t - option_price_estimate), min_valuation_error)
        num_iterations += 1
    
    # Check for convergence
    if abs(option_price_t - option_price_estimate) < precision:
        return implied_vol
    else:
        raise ValueError("Failed to converge with sufficient precision")

In [26]:
F_t = 50
X_t = 55
t = 1/12
r = 0.04
option_price_t = 6
precision = 0.0001
max_num_iterations = 100


implied_volatility_black76_european("call", F_t, X_t, t, r, option_price_t, precision, max_num_iterations)

INITIAL IMPLIED VOLATILITY = 1.4467829863581296


1.3797252282345918